In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.maze_functions.maze_generation import random_maze_generator, complete_maze
from src.maze_functions.feature_extractor import get_total_length, get_coord_list_plot, get_coord_list_matrix, get_no_corners, extract_features
import csv
import pandas as pd

In [ ]:
def pretty_print_maze(maze: np.ndarray):
    for row in maze:
        print(" ".join(str(cell).rjust(4) for cell in row))


def destruct_maze(maze: np.ndarray,
                  locations_in_maze: list[list[int]],
                  index_1: int,
                  index_2: int) -> tuple[np.ndarray, list[list[int]], np.ndarray]:
    
    sorted_indices = np.sort([index_1, index_2])

    deleted_locations_in_maze = locations_in_maze[sorted_indices[0]: sorted_indices[1]]

    deleted_maze = np.copy(maze)
    for X, Y, V in deleted_locations_in_maze:
        deleted_maze[X, Y] = 0

    return deleted_maze, deleted_locations_in_maze, sorted_indices
    
def repair_maze(deleted_maze: np.ndarray,
                old_maze: np.ndarray,
                locations_in_maze: list[list[int]],
                random_indices: np.ndarray,
                repair_mode: str) -> np.ndarray:

    LEFT_LOCATIONS = get_coord_list_matrix(deleted_maze)

    LAST_LOCATION_BEFORE_EMPTY = locations_in_maze[random_indices[0] - 1]

    TARGET_LOCATION = (locations_in_maze[random_indices[1]][0], locations_in_maze[random_indices[1]][1])
    START_COORDS = [LAST_LOCATION_BEFORE_EMPTY[0], LAST_LOCATION_BEFORE_EMPTY[1],
                    old_maze[LAST_LOCATION_BEFORE_EMPTY[0], LAST_LOCATION_BEFORE_EMPTY[1]].item()]

    NEW_MAZE = complete_maze(deleted_maze,
                             START_COORDS,
                             TARGET_LOCATION,
                             LEFT_LOCATIONS,
                             repair_mode)

    return NEW_MAZE


In [ ]:
step_size = 0.5
side_length = 20

N = 800
number_of_maze = 0
maze_library = {}

total_length_list = []

target_loc_modes = ["east", "north"]

for i in range(N):
    for target_loc_mode in target_loc_modes:
        number_of_maze += 1
        maze_matrix = random_maze_generator(side_length, target_loc_mode=target_loc_mode, path_finding_mode='random')
        coordinates = get_coord_list_plot(maze_matrix)
        
        # Multiply every coordinate by step size 
        coordinates = [(coord[0] * step_size, coord[1] * step_size) for coord in coordinates]
        matrix_coordinates = get_coord_list_matrix(maze_matrix)
        
        maze_id = f"Maze_{number_of_maze}"
        coordinates_str = '*'.join([f"{coord[0]} {coord[1]}" for coord in coordinates])
        
        # Extract features
        total_length = get_total_length(matrix_coordinates, step_size)
        no_corners = get_no_corners(matrix_coordinates)
        maze_step_size = 0.5
        maze_side_length = (step_size * side_length)
        
        # plot_maze(maze_matrix)
        
        # Store features
        maze_library[maze_id] = {
            "OutletPosition": target_loc_mode,
            "StepSize": maze_step_size,
            "SideLength": maze_side_length,
            "TotalLength": total_length,
            "Corner": no_corners,
            "Coordinates": coordinates_str
        }
        
               
        
with open('maze_library.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["MazeID", "OutletPosition", "StepSize", "SideLength", "TotalLength", "Corner", "Coordinates"])
    for key, value in maze_library.items():
        writer.writerow([key, value["OutletPosition"], value["StepSize"], value["SideLength"], value["TotalLength"], value["Corner"], value["Coordinates"]])
    
# load the library
maze_library = pd.read_csv('maze_library.csv')
maze_library

In [ ]:

# Build the longest maze for each target location
longest_east_maze_matrix = random_maze_generator(side_length, target_loc_mode='east', path_finding_mode='longest')
longest_north_maze_matrix = random_maze_generator(side_length, target_loc_mode='north', path_finding_mode='longest')

   
longest_maze_lib = {
    "east": longest_east_maze_matrix,
    "north": longest_north_maze_matrix
}

num_of_maze = 0
repair_maze_lib = {}

modes = ['shortest', 'random']

for target_loc_mode, maze_matrix in longest_maze_lib.items():
    
    longest_maze_matrix = longest_maze_lib[target_loc_mode]
    longest_matrix_coordinates = get_coord_list_matrix(longest_maze_matrix)
    
    largest_number_in_matrix = np.max(longest_maze_matrix)
        
    for i in range(2, largest_number_in_matrix):
        
        for mode in modes:
            
            # Destruct the path from tha chosen number to the largest number and construct a new path as the shortest path
            chosen_index = largest_number_in_matrix - i
            cut_maze_matrix, locations_in_cut_maze, sorted_cut_indices = destruct_maze(longest_maze_matrix,
                                                                                       longest_matrix_coordinates,
                                                                                       chosen_index,
                                                                                       largest_number_in_matrix-1)
            
                
            # Repair the maze
            repaired_maze_matrix = repair_maze(cut_maze_matrix,
                                               longest_maze_matrix,
                                               longest_matrix_coordinates,
                                               sorted_cut_indices,
                                               repair_mode=mode)
            
            num_of_maze += 1
            repaired_maze_step_size = 0.5
            repaired_maze_side_length = (step_size * side_length)
            repaired_total_length = get_total_length(get_coord_list_matrix(repaired_maze_matrix), step_size)
            repaired_no_corners = get_no_corners(get_coord_list_matrix(repaired_maze_matrix))
            repaired_coordinates = get_coord_list_plot(repaired_maze_matrix)
            
            # Multiply every coordinate by step size
            repaired_coordinates = [(coord[0] * step_size, coord[1] * repaired_maze_step_size) for coord in repaired_coordinates]
            repaired_coordinates_str = '*'.join([f"{coord[0]} {coord[1]}" for coord in repaired_coordinates])
            
            repair_maze_lib[f"Maze_{num_of_maze}"] = {
                "OutletPosition": target_loc_mode,
                "StepSize": repaired_maze_step_size,
                "SideLength": repaired_maze_side_length,
                "TotalLength": repaired_total_length,
                "Corner": repaired_no_corners,
                "Coordinates": repaired_coordinates_str
            }
            
            """plt.show()
            plot_maze(repaired_maze_matrix)
            plt.pause(0.5)"""
        
        
with open('repair_maze_library.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["MazeID", "OutletPosition", "StepSize", "SideLength", "TotalLength", "Corner", "Coordinates"])
    for key, value in repair_maze_lib.items():
        writer.writerow([key, value["OutletPosition"], value["StepSize"], value["SideLength"], value["TotalLength"], value["Corner"], value["Coordinates"]])
        
# load the library
repaired_maze_library = pd.read_csv('repair_maze_library.csv')
repaired_maze_library

In [ ]:
# merge the two libraries
merged_maze_library = pd.concat([maze_library, repaired_maze_library], ignore_index=True)
merged_maze_library

In [ ]:
# drop duplicates by StepSize, SideLength, TotalLength, Corner
merged_maze_library_no_duplicates = merged_maze_library.drop_duplicates(subset=["OutletPosition", "StepSize", "SideLength", "TotalLength", "Corner"])
merged_maze_library_no_duplicates.reset_index(drop=True, inplace=True)
merged_maze_library_no_duplicates

In [ ]:
# plot TotalLength graph histogram
plt.hist(merged_maze_library["TotalLength"], bins=100, color='blue', edgecolor='black')
plt.xlabel('Total Length')
plt.ylabel('Frequency')
plt.title('Total Length Histogram')
plt.show()
